In [1]:
import pynvml as nvml
import datetime 
import time

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [3]:
class DataWriter():
    def __init__(self, filename):
        self.filename = filename
        if self.filename == "":
            self.filename=self.getfileName()+".csv"

    def appendRecord(self, record):
        file = open(self.filename, 'a')
        file.write(record)
        file.write("\n")
        file.close()

    @staticmethod
    def addColumnToRecord(string):
        return string + ","

    @staticmethod
    def getDateTimeString():
        return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

    @staticmethod
    def getfileName():
        return datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [4]:
class NvidiaGPUMonitor():

    def __init__(self, name=""):
        self.filename = name
        try:
            nvml.nvmlShutdown()
        except nvml.NVMLError_Uninitialized, err:
            print("No existing instance found")
        try:
            nvml.nvmlInit()
            self.gpuCount = int(nvml.nvmlDeviceGetCount())
            
            self.dataWriter = DataWriter(self.filename)
            self._initializeData()
        except nvml.NVMLError, err:
            print("Failed to initialize NVML: ", err)
            print("Exiting...")

    def _initializeData(self):
        self.dataWriter.appendRecord("Time, GPU Index, Temp C, GPU %, Memory %")

    @staticmethod
    def closeConnection():
        try:
            nvml.nvmlShutdown()
        except nvml.VMLError, err:
            print("Error shutting down NVML:")
            

    @staticmethod
    def getTemperature(gpu):
        return str(nvml.nvmlDeviceGetTemperature(gpu, nvml.NVML_TEMPERATURE_GPU))

    @staticmethod
    def getUsedMemory(gpu):
        return str(nvml.nvmlDeviceGetUtilizationRates(gpu).memory)

    @staticmethod
    def getUsedCore(gpu):
        return str(nvml.nvmlDeviceGetUtilizationRates(gpu).gpu)



    def getDataPoint(self, gpuSelection):
        index = 0
        gpu = nvml.nvmlDeviceGetHandleByIndex(0)
        outputRecord = self.dataWriter.getDateTimeString() + ","+ "GPU"+str(index) +"," + self.getTemperature(gpu) + ',' +self.getUsedCore(gpu) + ',' +self.getUsedMemory(gpu)

        self.dataWriter.appendRecord(outputRecord)


In [5]:
def main():
    gpu = NvidiaGPUMonitor()
    # Create headers, would like to do this automatically later
    print("Collecting data...")
    while(True):
        time.sleep(2)
        gpu.getDataPoint(0)

try:
    main()
except KeyboardInterrupt:
    
    print("Done Collecting...")
    
    #Graph some Data

No existing instance found
Done Collecting...
